# Argumentation framework

In [1]:
import sys
import os

sys.path.append(os.path.join(
    os.getcwd(),
    "utils",
))

In [2]:
from utils.stable import gen as gen_stable
from utils.grounded import gen as gen_grounded

g_stable = gen_stable(100, 0, 10, 0, 10, 0, 10, .3)
g_stable.save("stable.txt")

g_grounded = gen_grounded(500, .3)
g_grounded.save("grounded.txt")

Initialised _ReproducibleRandom with seed 164569603467


In [3]:
from utils.solve import solve
from utils.benchmark import benchmark

benchmark(solve, "stable.txt", "SE-ST")

0.03129720687866211